## Slicing, combining, and exporting water column features for O2-based productivity estimates

### Upstream:

#### - Started with using python-CTD to convert 1 m binned CTD files (Seabird cnv) to a single csv for each station 
#### - Downloaded Al's 'ave' cnv files from the POMZ shared drive

### Downstream:

#### - Will take times and O2 concentrations to estimate GOP and CR in the mixed layer, the primary chlorophyll max, and the deep chlorophyll max using MATLAB sine fitting script from Barone et al., 2019.

### Station 33/P3, April 2018 (Revelle, RR1804):

In [1]:
import io
import requests
from pathlib import Path
import seaborn as sns
import pandas as pd

import ctd
import datetime as dt

In [2]:
# Little script to convert datetime to MATLAB datenum format to go into Barone ML script later

def datetime_to_datenum(dt_obj):
    matlab_datenum = dt_obj.toordinal() + 366 + (dt_obj - dt.datetime(2000, 1, 1)).total_seconds() / 86400
    return matlab_datenum

In [3]:
# Test how the datenum conversion works
now = dt.datetime.now()  # get current datetime
matlab_datenum = datetime_to_datenum(now)
print(matlab_datenum)
print(now)

747490.8211847276
2023-04-12 19:42:30.360465


In [4]:
# Define the starting datetimes of each cast

st59 = '2018-04-21 15:39:15'
st60 = '2018-04-22 10:12:45'
st61 = '2018-04-22 14:04:25'
st62 = '2018-04-22 22:00:31'
st63 = '2018-04-23 14:00:59'
st64 = '2018-04-24 14:03:01'
st65 = '2018-04-24 21:04:47'
st66 = '2018-04-25 15:02:53'
st67 = '2018-04-26 00:07:04'
st68 = '2018-04-26 06:06:44'
st69 = '2018-04-26 12:04:27'
st70 = '2018-04-26 18:56:09'
st71 = '2018-04-27 00:01:50'
st72 = '2018-04-27 09:44:07'

### Cell structure

1.  Read from the individual cast cnv
2.  Add the correct and pertinent start time to each downcast
3.  Slice the mixed layer and save as a csv in RR1804/P1_2018/mixed-layer/
4.  Integrate the O2, beam transmission, and chlorophyll fluoresence over the mixed layer and save each as a variable
5.  save a new dataframe with the lat, long, StartTime, and integrated value variables

6.  Repeat steps 3-5 for the PCM and DCM

7. Combine all cast integration dataframes for each water column feature and save as a csv in RR1804/P1_2018/integrations/

In [9]:
# read from cnv with 'ignore' added to depSM
cast59 = ctd.from_cnv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/RR1804/RR1803-St33-P1/Station33_cast59.cnv")

# set up datetime to datenum 
ST_59 = pd.to_datetime(st59)

# split up and down casts; we want the downcast for the profile
down59, up59 = cast59.split()

# add a column with the start date and time in UTC (this is from the cnv header or from ship's log)
down59.insert(0, 'StartTime', pd.to_datetime(st59).replace(microsecond=0))

# slice the mixed layer out and save as csv
p1_59_ml = down59[down59['sigma-�00'].between(22.0,23.6)]
p1_59_ml.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/RR1804/P1_2018/mixed_layer/Station33_cast59_ML.csv")

# get the sum of mixed layer slice oxygen, beam transmission, and chlorophyll fluor
cf_sum_p1_59_ml = p1_59_ml['flSP'].sum()
print('ML Chl fluor sum:', cf_sum_p1_59_ml)

bt_sum_p1_59_ml = p1_59_ml['CStarTr0'].sum()
print('ML Beam trans sum:', bt_sum_p1_59_ml)

o2_sum_p1_59_ml = p1_59_ml['sbox0Mm/Kg'].sum()
print('ML O2 Mm/Kg sum:', o2_sum_p1_59_ml)

# Make lists of the input for the ML integrated dataframe
ML_data_59 = {'Cast':[59], 'StartTime':st59, 'StartTime matlab datenum': datetime_to_datenum(ST_59), \
           'Int chl fluor': cf_sum_p1_59_ml, \
           'Int beam trans': bt_sum_p1_59_ml, \
            'Int O2 Mm/Kg': o2_sum_p1_59_ml}

# slice the primary chlorophyll max out and save as csv
p1_59_pcm = down59[down59['sigma-�00'].between(23.6,25.3)]
p1_59_pcm.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/RR1804/P1_2018/primary_chl/Station33_cast59_PCM.csv")

# get the sum of primary chl slice oxygen and beam transmission
cf_sum_p1_59_pcm = p1_59_pcm['flSP'].sum()
print('PCM Chl fluor sum:', cf_sum_p1_59_pcm)

bt_sum_p1_59_pcm = p1_59_pcm['CStarTr0'].sum()
print('PCM Beam trans sum:', bt_sum_p1_59_pcm)

o2_sum_p1_59_pcm = p1_59_pcm['sbox0Mm/Kg'].sum()
print('PCM O2 Mm/Kg sum:', o2_sum_p1_59_pcm)

# Make lists of the input for the PCM integrated dataframe
PCM_data_59 = {'Cast':[59], 'StartTime':st59, 'StartTime matlab datenum': datetime_to_datenum(ST_59), \
            'Int chl fluor': cf_sum_p1_59_pcm, \
           'Int beam trans': bt_sum_p1_59_pcm, \
            'Int O2 Mm/Kg': o2_sum_p1_59_pcm}

# slice the deep chlorophyll max out and save as csv
p1_59_dcm = down59[down59['sigma-�00'].between(25.3,26.4)]
p1_59_dcm.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/RR1804/P1_2018/deep_chl/Station33_cast59_DCM.csv")

# get the sum of deep chl slice oxygen and beam transmission
cf_sum_p1_59_dcm = p1_59_dcm['flSP'].sum()
print('DCM Chl fluor sum:', cf_sum_p1_59_dcm)

bt_sum_p1_59_dcm = p1_59_dcm['CStarTr0'].sum()
print('DCM Beam trans sum:', bt_sum_p1_59_dcm)

o2_sum_p1_59_dcm = p1_59_dcm['sbox0Mm/Kg'].sum()
print('DCM O2 Mm/Kg sum:', o2_sum_p1_59_dcm)

# Make lists of the input for the DCM integrated dataframe
DCM_data_59 = {'Cast':[59], 'StartTime':st59, 'StartTime matlab datenum': datetime_to_datenum(ST_59), \
            'Int chl fluor': cf_sum_p1_59_dcm, \
           'Int beam trans': bt_sum_p1_59_dcm, \
            'Int O2 Mm/Kg': o2_sum_p1_59_dcm}

ML Chl fluor sum: 0.0
ML Beam trans sum: 0.0
ML O2 Mm/Kg sum: 0.0
PCM Chl fluor sum: 61.12622999999999
PCM Beam trans sum: 112.69470000000001
PCM O2 Mm/Kg sum: 2577.2370000000005
DCM Chl fluor sum: 43.56976900000001
DCM Beam trans sum: 722.4413999999999
DCM O2 Mm/Kg sum: 218.65699999999998


In [11]:
# Make lists into a new dataframe called ML_2018_p1
ML_2018_p1 = pd.DataFrame(ML_data_59)

# Same for PCM
PCM_2018_p1 = pd.DataFrame(PCM_data_59)

# And DCM
DCM_2018_p1 = pd.DataFrame(DCM_data_59)

DCM_2018_p1.head()

,Cast,StartTime,StartTime matlab datenum,Int chl fluor,Int beam trans,Int O2 Mm/Kg
0,59,2018-04-21 15:39:15,743856.652257,43.569769,722.4414,218.657


In [12]:
# read from cnv with 'ignore' added to depSM
cast60 = ctd.from_cnv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/RR1804/RR1803-St33-P1/Station33_cast60.cnv")

# set up datetime to datenum 
ST_60 = pd.to_datetime(st60)

# split up and down casts; we want the downcast for the profile
down60, up60 = cast60.split()

# add a column with the start date and time in UTC (this is from the cnv header or from ship's log)
down60.insert(0, 'StartTime', pd.to_datetime(st60).replace(microsecond=0))

# slice the mixed layer out and save as csv
p1_60_ml = down60[down60['sigma-�00'].between(22.0,23.6)]
p1_60_ml.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/RR1804/P1_2018/mixed_layer/Station33_cast60_ML.csv")

# get the sum of mixed layer slice oxygen, beam transmission, and chlorophyll fluor
cf_sum_p1_60_ml = p1_60_ml['flSP'].sum()
print('ML Chl fluor sum:', cf_sum_p1_60_ml)

bt_sum_p1_60_ml = p1_60_ml['CStarTr0'].sum()
print('ML Beam trans sum:', bt_sum_p1_60_ml)

o2_sum_p1_60_ml = p1_60_ml['sbox0Mm/Kg'].sum()
print('ML O2 Mm/Kg sum:', o2_sum_p1_60_ml)

# Make lists of the input for the ML integrated dataframe
ML_data_60 = {'Cast':[60], 'StartTime':st60, 'StartTime matlab datenum': datetime_to_datenum(ST_60), \
           'Int chl fluor': cf_sum_p1_60_ml, \
           'Int beam trans': bt_sum_p1_60_ml, \
            'Int O2 Mm/Kg': o2_sum_p1_60_ml}

# slice the primary chlorophyll max out and save as csv
p1_60_pcm = down60[down60['sigma-�00'].between(23.6,25.3)]
p1_60_pcm.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/RR1804/P1_2018/primary_chl/Station33_cast60_PCM.csv")

# get the sum of primary chl slice oxygen and beam transmission
cf_sum_p1_60_pcm = p1_60_pcm['flSP'].sum()
print('PCM Chl fluor sum:', cf_sum_p1_60_pcm)

bt_sum_p1_60_pcm = p1_60_pcm['CStarTr0'].sum()
print('PCM Beam trans sum:', bt_sum_p1_60_pcm)

o2_sum_p1_60_pcm = p1_60_pcm['sbox0Mm/Kg'].sum()
print('PCM O2 Mm/Kg sum:', o2_sum_p1_60_pcm)

# Make lists of the input for the PCM integrated dataframe
PCM_data_60 = {'Cast':[60], 'StartTime':st60, 'StartTime matlab datenum': datetime_to_datenum(ST_60), \
            'Int chl fluor': cf_sum_p1_60_pcm, \
           'Int beam trans': bt_sum_p1_60_pcm, \
            'Int O2 Mm/Kg': o2_sum_p1_60_pcm}

# slice the deep chlorophyll max out and save as csv
p1_60_dcm = down60[down60['sigma-�00'].between(25.3,26.4)]
p1_60_dcm.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/RR1804/P1_2018/deep_chl/Station33_cast60_DCM.csv")

# get the sum of deep chl slice oxygen and beam transmission
cf_sum_p1_60_dcm = p1_60_dcm['flSP'].sum()
print('DCM Chl fluor sum:', cf_sum_p1_60_dcm)

bt_sum_p1_60_dcm = p1_60_dcm['CStarTr0'].sum()
print('DCM Beam trans sum:', bt_sum_p1_60_dcm)

o2_sum_p1_60_dcm = p1_60_dcm['sbox0Mm/Kg'].sum()
print('DCM O2 Mm/Kg sum:', o2_sum_p1_60_dcm)

# Make lists of the input for the DCM integrated dataframe
DCM_data_60 = {'Cast':[60], 'StartTime':st60, 'StartTime matlab datenum': datetime_to_datenum(ST_60), \
            'Int chl fluor': cf_sum_p1_60_dcm, \
           'Int beam trans': bt_sum_p1_60_dcm, \
            'Int O2 Mm/Kg': o2_sum_p1_60_dcm}

ML Chl fluor sum: 13.591000000000001
ML Beam trans sum: 43.1305
ML O2 Mm/Kg sum: 1916.1639999999998
PCM Chl fluor sum: 29.768699999999995
PCM Beam trans sum: 141.6178
PCM O2 Mm/Kg sum: 2662.933
DCM Chl fluor sum: 49.582895
DCM Beam trans sum: 494.22339999999997
DCM O2 Mm/Kg sum: 152.554


In [13]:
# read from cnv with 'ignore' added to depSM
cast61 = ctd.from_cnv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/RR1804/RR1803-St33-P1/Station33_cast61.cnv")

# set up datetime to datenum 
ST_61 = pd.to_datetime(st61)

# split up and down casts; we want the downcast for the profile
down61, up61 = cast61.split()

# add a column with the start date and time in UTC (this is from the cnv header or from ship's log)
down61.insert(0, 'StartTime', pd.to_datetime(st61).replace(microsecond=0))

# slice the mixed layer out and save as csv
p1_61_ml = down61[down61['sigma-�00'].between(22.0,23.6)]
p1_61_ml.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/RR1804/P1_2018/mixed_layer/Station33_cast61_ML.csv")

# get the sum of mixed layer slice oxygen, beam transmission, and chlorophyll fluor
cf_sum_p1_61_ml = p1_61_ml['flSP'].sum()
print('ML Chl fluor sum:', cf_sum_p1_61_ml)

bt_sum_p1_61_ml = p1_61_ml['CStarTr0'].sum()
print('ML Beam trans sum:', bt_sum_p1_61_ml)

o2_sum_p1_61_ml = p1_61_ml['sbox0Mm/Kg'].sum()
print('ML O2 Mm/Kg sum:', o2_sum_p1_61_ml)

# Make lists of the input for the ML integrated dataframe
ML_data_61 = {'Cast':[61], 'StartTime':st61, 'StartTime matlab datenum': datetime_to_datenum(ST_61), \
           'Int chl fluor': cf_sum_p1_61_ml, \
           'Int beam trans': bt_sum_p1_61_ml, \
            'Int O2 Mm/Kg': o2_sum_p1_61_ml}

# slice the primary chlorophyll max out and save as csv
p1_61_pcm = down61[down61['sigma-�00'].between(23.6,25.3)]
p1_61_pcm.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/RR1804/P1_2018/primary_chl/Station33_cast61_PCM.csv")

# get the sum of primary chl slice oxygen and beam transmission
cf_sum_p1_61_pcm = p1_61_pcm['flSP'].sum()
print('PCM Chl fluor sum:', cf_sum_p1_61_pcm)

bt_sum_p1_61_pcm = p1_61_pcm['CStarTr0'].sum()
print('PCM Beam trans sum:', bt_sum_p1_61_pcm)

o2_sum_p1_61_pcm = p1_61_pcm['sbox0Mm/Kg'].sum()
print('PCM O2 Mm/Kg sum:', o2_sum_p1_61_pcm)

# Make lists of the input for the PCM integrated dataframe
PCM_data_61 = {'Cast':[61], 'StartTime':st61, 'StartTime matlab datenum': datetime_to_datenum(ST_61), \
            'Int chl fluor': cf_sum_p1_61_pcm, \
           'Int beam trans': bt_sum_p1_61_pcm, \
            'Int O2 Mm/Kg': o2_sum_p1_61_pcm}

# slice the deep chlorophyll max out and save as csv
p1_61_dcm = down61[down61['sigma-�00'].between(25.3,26.4)]
p1_61_dcm.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/RR1804/P1_2018/deep_chl/Station33_cast61_DCM.csv")

# get the sum of deep chl slice oxygen and beam transmission
cf_sum_p1_61_dcm = p1_61_dcm['flSP'].sum()
print('DCM Chl fluor sum:', cf_sum_p1_61_dcm)

bt_sum_p1_61_dcm = p1_61_dcm['CStarTr0'].sum()
print('DCM Beam trans sum:', bt_sum_p1_61_dcm)

o2_sum_p1_61_dcm = p1_61_dcm['sbox0Mm/Kg'].sum()
print('DCM O2 Mm/Kg sum:', o2_sum_p1_61_dcm)

# Make lists of the input for the DCM integrated dataframe
DCM_data_61 = {'Cast':[61], 'StartTime':st61, 'StartTime matlab datenum': datetime_to_datenum(ST_61), \
            'Int chl fluor': cf_sum_p1_61_dcm, \
           'Int beam trans': bt_sum_p1_61_dcm, \
            'Int O2 Mm/Kg': o2_sum_p1_61_dcm}

ML Chl fluor sum: 34.9435
ML Beam trans sum: 76.215
ML O2 Mm/Kg sum: 3480.805
PCM Chl fluor sum: 15.67199
PCM Beam trans sum: 75.1618
PCM O2 Mm/Kg sum: 1328.6319999999998
DCM Chl fluor sum: 56.585747000000005
DCM Beam trans sum: 648.3669
DCM O2 Mm/Kg sum: 246.46599999999995


In [14]:
# read from cnv with 'ignore' added to depSM
cast62 = ctd.from_cnv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/RR1804/RR1803-St33-P1/Station33_cast62.cnv")

# set up datetime to datenum 
ST_62 = pd.to_datetime(st62)

# split up and down casts; we want the downcast for the profile
down62, up62 = cast62.split()

# add a column with the start date and time in UTC (this is from the cnv header or from ship's log)
down62.insert(0, 'StartTime', pd.to_datetime(st62).replace(microsecond=0))

# slice the mixed layer out and save as csv
p1_62_ml = down62[down62['sigma-�00'].between(22.0,23.6)]
p1_62_ml.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/RR1804/P1_2018/mixed_layer/Station33_cast62_ML.csv")

# get the sum of mixed layer slice oxygen, beam transmission, and chlorophyll fluor
cf_sum_p1_62_ml = p1_62_ml['flSP'].sum()
print('ML Chl fluor sum:', cf_sum_p1_62_ml)

bt_sum_p1_62_ml = p1_62_ml['CStarTr0'].sum()
print('ML Beam trans sum:', bt_sum_p1_62_ml)

o2_sum_p1_62_ml = p1_62_ml['sbox0Mm/Kg'].sum()
print('ML O2 Mm/Kg sum:', o2_sum_p1_62_ml)

# Make lists of the input for the ML integrated dataframe
ML_data_62 = {'Cast':[62], 'StartTime':st62, 'StartTime matlab datenum': datetime_to_datenum(ST_62), \
           'Int chl fluor': cf_sum_p1_62_ml, \
           'Int beam trans': bt_sum_p1_62_ml, \
            'Int O2 Mm/Kg': o2_sum_p1_62_ml}

# slice the primary chlorophyll max out and save as csv
p1_62_pcm = down62[down62['sigma-�00'].between(23.6,25.3)]
p1_62_pcm.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/RR1804/P1_2018/primary_chl/Station33_cast62_PCM.csv")

# get the sum of primary chl slice oxygen and beam transmission
cf_sum_p1_62_pcm = p1_62_pcm['flSP'].sum()
print('PCM Chl fluor sum:', cf_sum_p1_62_pcm)

bt_sum_p1_62_pcm = p1_62_pcm['CStarTr0'].sum()
print('PCM Beam trans sum:', bt_sum_p1_62_pcm)

o2_sum_p1_62_pcm = p1_62_pcm['sbox0Mm/Kg'].sum()
print('PCM O2 Mm/Kg sum:', o2_sum_p1_62_pcm)

# Make lists of the input for the PCM integrated dataframe
PCM_data_62 = {'Cast':[62], 'StartTime':st62, 'StartTime matlab datenum': datetime_to_datenum(ST_62), \
            'Int chl fluor': cf_sum_p1_62_pcm, \
           'Int beam trans': bt_sum_p1_62_pcm, \
            'Int O2 Mm/Kg': o2_sum_p1_62_pcm}

# slice the deep chlorophyll max out and save as csv
p1_62_dcm = down62[down62['sigma-�00'].between(25.3,26.4)]
p1_62_dcm.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/RR1804/P1_2018/deep_chl/Station33_cast62_DCM.csv")

# get the sum of deep chl slice oxygen and beam transmission
cf_sum_p1_62_dcm = p1_62_dcm['flSP'].sum()
print('DCM Chl fluor sum:', cf_sum_p1_62_dcm)

bt_sum_p1_62_dcm = p1_62_dcm['CStarTr0'].sum()
print('DCM Beam trans sum:', bt_sum_p1_62_dcm)

o2_sum_p1_62_dcm = p1_62_dcm['sbox0Mm/Kg'].sum()
print('DCM O2 Mm/Kg sum:', o2_sum_p1_62_dcm)

# Make lists of the input for the DCM integrated dataframe
DCM_data_62 = {'Cast':[62], 'StartTime':st62, 'StartTime matlab datenum': datetime_to_datenum(ST_62), \
            'Int chl fluor': cf_sum_p1_62_dcm, \
           'Int beam trans': bt_sum_p1_62_dcm, \
            'Int O2 Mm/Kg': o2_sum_p1_62_dcm}

ML Chl fluor sum: 20.381899999999998
ML Beam trans sum: 41.7564
ML O2 Mm/Kg sum: 1928.755
PCM Chl fluor sum: 29.96295
PCM Beam trans sum: 109.6267
PCM O2 Mm/Kg sum: 2251.013
DCM Chl fluor sum: 51.930882
DCM Beam trans sum: 662.4409
DCM O2 Mm/Kg sum: 243.89


In [15]:
# read from cnv with 'ignore' added to depSM
cast63 = ctd.from_cnv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/RR1804/RR1803-St33-P1/Station33_cast63.cnv")

# set up datetime to datenum 
ST_63 = pd.to_datetime(st63)

# split up and down casts; we want the downcast for the profile
down63, up63 = cast63.split()

# add a column with the start date and time in UTC (this is from the cnv header or from ship's log)
down63.insert(0, 'StartTime', pd.to_datetime(st63).replace(microsecond=0))

# slice the mixed layer out and save as csv
p1_63_ml = down63[down63['sigma-�00'].between(22.0,23.6)]
p1_63_ml.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/RR1804/P1_2018/mixed_layer/Station33_cast63_ML.csv")

# get the sum of mixed layer slice oxygen, beam transmission, and chlorophyll fluor
cf_sum_p1_63_ml = p1_63_ml['flSP'].sum()
print('ML Chl fluor sum:', cf_sum_p1_63_ml)

bt_sum_p1_63_ml = p1_63_ml['CStarTr0'].sum()
print('ML Beam trans sum:', bt_sum_p1_63_ml)

o2_sum_p1_63_ml = p1_63_ml['sbox0Mm/Kg'].sum()
print('ML O2 Mm/Kg sum:', o2_sum_p1_63_ml)

# Make lists of the input for the ML integrated dataframe
ML_data_63 = {'Cast':[63], 'StartTime':st63, 'StartTime matlab datenum': datetime_to_datenum(ST_63), \
           'Int chl fluor': cf_sum_p1_63_ml, \
           'Int beam trans': bt_sum_p1_63_ml, \
            'Int O2 Mm/Kg': o2_sum_p1_63_ml}

# slice the primary chlorophyll max out and save as csv
p1_63_pcm = down63[down63['sigma-�00'].between(23.6,25.3)]
p1_63_pcm.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/RR1804/P1_2018/primary_chl/Station33_cast63_PCM.csv")

# get the sum of primary chl slice oxygen and beam transmission
cf_sum_p1_63_pcm = p1_63_pcm['flSP'].sum()
print('PCM Chl fluor sum:', cf_sum_p1_63_pcm)

bt_sum_p1_63_pcm = p1_63_pcm['CStarTr0'].sum()
print('PCM Beam trans sum:', bt_sum_p1_63_pcm)

o2_sum_p1_63_pcm = p1_63_pcm['sbox0Mm/Kg'].sum()
print('PCM O2 Mm/Kg sum:', o2_sum_p1_63_pcm)

# Make lists of the input for the PCM integrated dataframe
PCM_data_63 = {'Cast':[63], 'StartTime':st63, 'StartTime matlab datenum': datetime_to_datenum(ST_63), \
            'Int chl fluor': cf_sum_p1_63_pcm, \
           'Int beam trans': bt_sum_p1_63_pcm, \
            'Int O2 Mm/Kg': o2_sum_p1_63_pcm}

# slice the deep chlorophyll max out and save as csv
p1_63_dcm = down63[down63['sigma-�00'].between(25.3,26.4)]
p1_63_dcm.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/RR1804/P1_2018/deep_chl/Station33_cast63_DCM.csv")

# get the sum of deep chl slice oxygen and beam transmission
cf_sum_p1_63_dcm = p1_63_dcm['flSP'].sum()
print('DCM Chl fluor sum:', cf_sum_p1_63_dcm)

bt_sum_p1_63_dcm = p1_63_dcm['CStarTr0'].sum()
print('DCM Beam trans sum:', bt_sum_p1_63_dcm)

o2_sum_p1_63_dcm = p1_63_dcm['sbox0Mm/Kg'].sum()
print('DCM O2 Mm/Kg sum:', o2_sum_p1_63_dcm)

# Make lists of the input for the DCM integrated dataframe
DCM_data_63 = {'Cast':[63], 'StartTime':st63, 'StartTime matlab datenum': datetime_to_datenum(ST_63), \
            'Int chl fluor': cf_sum_p1_63_dcm, \
           'Int beam trans': bt_sum_p1_63_dcm, \
            'Int O2 Mm/Kg': o2_sum_p1_63_dcm}

ML Chl fluor sum: 15.83736
ML Beam trans sum: 47.4445
ML O2 Mm/Kg sum: 2159.6789999999996
PCM Chl fluor sum: 17.77299
PCM Beam trans sum: 93.3572
PCM O2 Mm/Kg sum: 1393.82
DCM Chl fluor sum: 51.863331
DCM Beam trans sum: 662.4267
DCM O2 Mm/Kg sum: 214.067


In [16]:
# read from cnv with 'ignore' added to depSM
cast64 = ctd.from_cnv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/RR1804/RR1803-St33-P1/Station33_cast64.cnv")

# set up datetime to datenum 
ST_64 = pd.to_datetime(st64)

# split up and down casts; we want the downcast for the profile
down64, up64 = cast64.split()

# add a column with the start date and time in UTC (this is from the cnv header or from ship's log)
down64.insert(0, 'StartTime', pd.to_datetime(st64).replace(microsecond=0))

# slice the mixed layer out and save as csv
p1_64_ml = down64[down64['sigma-�00'].between(22.0,23.6)]
p1_64_ml.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/RR1804/P1_2018/mixed_layer/Station33_cast64_ML.csv")

# get the sum of mixed layer slice oxygen, beam transmission, and chlorophyll fluor
cf_sum_p1_64_ml = p1_64_ml['flSP'].sum()
print('ML Chl fluor sum:', cf_sum_p1_64_ml)

bt_sum_p1_64_ml = p1_64_ml['CStarTr0'].sum()
print('ML Beam trans sum:', bt_sum_p1_64_ml)

o2_sum_p1_64_ml = p1_64_ml['sbox0Mm/Kg'].sum()
print('ML O2 Mm/Kg sum:', o2_sum_p1_64_ml)

# Make lists of the input for the ML integrated dataframe
ML_data_64 = {'Cast':[64], 'StartTime':st64, 'StartTime matlab datenum': datetime_to_datenum(ST_64), \
           'Int chl fluor': cf_sum_p1_64_ml, \
           'Int beam trans': bt_sum_p1_64_ml, \
            'Int O2 Mm/Kg': o2_sum_p1_64_ml}

# slice the primary chlorophyll max out and save as csv
p1_64_pcm = down64[down64['sigma-�00'].between(23.6,25.3)]
p1_64_pcm.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/RR1804/P1_2018/primary_chl/Station33_cast64_PCM.csv")

# get the sum of primary chl slice oxygen and beam transmission
cf_sum_p1_64_pcm = p1_64_pcm['flSP'].sum()
print('PCM Chl fluor sum:', cf_sum_p1_64_pcm)

bt_sum_p1_64_pcm = p1_64_pcm['CStarTr0'].sum()
print('PCM Beam trans sum:', bt_sum_p1_64_pcm)

o2_sum_p1_64_pcm = p1_64_pcm['sbox0Mm/Kg'].sum()
print('PCM O2 Mm/Kg sum:', o2_sum_p1_64_pcm)

# Make lists of the input for the PCM integrated dataframe
PCM_data_64 = {'Cast':[64], 'StartTime':st64, 'StartTime matlab datenum': datetime_to_datenum(ST_64), \
            'Int chl fluor': cf_sum_p1_64_pcm, \
           'Int beam trans': bt_sum_p1_64_pcm, \
            'Int O2 Mm/Kg': o2_sum_p1_64_pcm}

# slice the deep chlorophyll max out and save as csv
p1_64_dcm = down64[down64['sigma-�00'].between(25.3,26.4)]
p1_64_dcm.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/RR1804/P1_2018/deep_chl/Station33_cast64_DCM.csv")

# get the sum of deep chl slice oxygen and beam transmission
cf_sum_p1_64_dcm = p1_64_dcm['flSP'].sum()
print('DCM Chl fluor sum:', cf_sum_p1_64_dcm)

bt_sum_p1_64_dcm = p1_64_dcm['CStarTr0'].sum()
print('DCM Beam trans sum:', bt_sum_p1_64_dcm)

o2_sum_p1_64_dcm = p1_64_dcm['sbox0Mm/Kg'].sum()
print('DCM O2 Mm/Kg sum:', o2_sum_p1_64_dcm)

# Make lists of the input for the DCM integrated dataframe
DCM_data_64 = {'Cast':[64], 'StartTime':st64, 'StartTime matlab datenum': datetime_to_datenum(ST_64), \
            'Int chl fluor': cf_sum_p1_64_dcm, \
           'Int beam trans': bt_sum_p1_64_dcm, \
            'Int O2 Mm/Kg': o2_sum_p1_64_dcm}

ML Chl fluor sum: 18.954890000000002
ML Beam trans sum: 100.66230000000002
ML O2 Mm/Kg sum: 4508.089999999999
PCM Chl fluor sum: 15.616409999999998
PCM Beam trans sum: 88.9701
PCM O2 Mm/Kg sum: 1464.1919999999996
DCM Chl fluor sum: 30.980209000000002
DCM Beam trans sum: 631.2198999999999
DCM O2 Mm/Kg sum: 392.11400000000003


In [17]:
# read from cnv with 'ignore' added to depSM
cast65 = ctd.from_cnv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/RR1804/RR1803-St33-P1/Station33_cast65.cnv")

# set up datetime to datenum 
ST_65 = pd.to_datetime(st65)

# split up and down casts; we want the downcast for the profile
down65, up65 = cast65.split()

# add a column with the start date and time in UTC (this is from the cnv header or from ship's log)
down65.insert(0, 'StartTime', pd.to_datetime(st65).replace(microsecond=0))

# slice the mixed layer out and save as csv
p1_65_ml = down65[down65['sigma-�00'].between(22.0,23.6)]
p1_65_ml.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/RR1804/P1_2018/mixed_layer/Station33_cast65_ML.csv")

# get the sum of mixed layer slice oxygen, beam transmission, and chlorophyll fluor
cf_sum_p1_65_ml = p1_65_ml['flSP'].sum()
print('ML Chl fluor sum:', cf_sum_p1_65_ml)

bt_sum_p1_65_ml = p1_65_ml['CStarTr0'].sum()
print('ML Beam trans sum:', bt_sum_p1_65_ml)

o2_sum_p1_65_ml = p1_65_ml['sbox0Mm/Kg'].sum()
print('ML O2 Mm/Kg sum:', o2_sum_p1_65_ml)

# Make lists of the input for the ML integrated dataframe
ML_data_65 = {'Cast':[65], 'StartTime':st65, 'StartTime matlab datenum': datetime_to_datenum(ST_65), \
           'Int chl fluor': cf_sum_p1_65_ml, \
           'Int beam trans': bt_sum_p1_65_ml, \
            'Int O2 Mm/Kg': o2_sum_p1_65_ml}

# slice the primary chlorophyll max out and save as csv
p1_65_pcm = down65[down65['sigma-�00'].between(23.6,25.3)]
p1_65_pcm.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/RR1804/P1_2018/primary_chl/Station33_cast65_PCM.csv")

# get the sum of primary chl slice oxygen and beam transmission
cf_sum_p1_65_pcm = p1_65_pcm['flSP'].sum()
print('PCM Chl fluor sum:', cf_sum_p1_65_pcm)

bt_sum_p1_65_pcm = p1_65_pcm['CStarTr0'].sum()
print('PCM Beam trans sum:', bt_sum_p1_65_pcm)

o2_sum_p1_65_pcm = p1_65_pcm['sbox0Mm/Kg'].sum()
print('PCM O2 Mm/Kg sum:', o2_sum_p1_65_pcm)

# Make lists of the input for the PCM integrated dataframe
PCM_data_65 = {'Cast':[65], 'StartTime':st65, 'StartTime matlab datenum': datetime_to_datenum(ST_65), \
            'Int chl fluor': cf_sum_p1_65_pcm, \
           'Int beam trans': bt_sum_p1_65_pcm, \
            'Int O2 Mm/Kg': o2_sum_p1_65_pcm}

# slice the deep chlorophyll max out and save as csv
p1_65_dcm = down65[down65['sigma-�00'].between(25.3,26.4)]
p1_65_dcm.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/RR1804/P1_2018/deep_chl/Station33_cast65_DCM.csv")

# get the sum of deep chl slice oxygen and beam transmission
cf_sum_p1_65_dcm = p1_65_dcm['flSP'].sum()
print('DCM Chl fluor sum:', cf_sum_p1_65_dcm)

bt_sum_p1_65_dcm = p1_65_dcm['CStarTr0'].sum()
print('DCM Beam trans sum:', bt_sum_p1_65_dcm)

o2_sum_p1_65_dcm = p1_65_dcm['sbox0Mm/Kg'].sum()
print('DCM O2 Mm/Kg sum:', o2_sum_p1_65_dcm)

# Make lists of the input for the DCM integrated dataframe
DCM_data_65 = {'Cast':[65], 'StartTime':st65, 'StartTime matlab datenum': datetime_to_datenum(ST_65), \
            'Int chl fluor': cf_sum_p1_65_dcm, \
           'Int beam trans': bt_sum_p1_65_dcm, \
            'Int O2 Mm/Kg': o2_sum_p1_65_dcm}

ML Chl fluor sum: 17.39061
ML Beam trans sum: 83.02
ML O2 Mm/Kg sum: 3700.612
PCM Chl fluor sum: 24.460990000000002
PCM Beam trans sum: 93.0222
PCM O2 Mm/Kg sum: 1453.7720000000002
DCM Chl fluor sum: 41.555736
DCM Beam trans sum: 675.7710000000002
DCM O2 Mm/Kg sum: 222.70099999999996


In [18]:
# read from cnv with 'ignore' added to depSM
cast66 = ctd.from_cnv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/RR1804/RR1803-St33-P1/Station33_cast66.cnv")

# set up datetime to datenum 
ST_66 = pd.to_datetime(st66)

# split up and down casts; we want the downcast for the profile
down66, up66 = cast66.split()

# add a column with the start date and time in UTC (this is from the cnv header or from ship's log)
down66.insert(0, 'StartTime', pd.to_datetime(st66).replace(microsecond=0))

# slice the mixed layer out and save as csv
p1_66_ml = down66[down66['sigma-�00'].between(22.0,23.6)]
p1_66_ml.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/RR1804/P1_2018/mixed_layer/Station33_cast66_ML.csv")

# get the sum of mixed layer slice oxygen, beam transmission, and chlorophyll fluor
cf_sum_p1_66_ml = p1_66_ml['flSP'].sum()
print('ML Chl fluor sum:', cf_sum_p1_66_ml)

bt_sum_p1_66_ml = p1_66_ml['CStarTr0'].sum()
print('ML Beam trans sum:', bt_sum_p1_66_ml)

o2_sum_p1_66_ml = p1_66_ml['sbox0Mm/Kg'].sum()
print('ML O2 Mm/Kg sum:', o2_sum_p1_66_ml)

# Make lists of the input for the ML integrated dataframe
ML_data_66 = {'Cast':[66], 'StartTime':st66, 'StartTime matlab datenum': datetime_to_datenum(ST_66), \
           'Int chl fluor': cf_sum_p1_66_ml, \
           'Int beam trans': bt_sum_p1_66_ml, \
            'Int O2 Mm/Kg': o2_sum_p1_66_ml}

# slice the primary chlorophyll max out and save as csv
p1_66_pcm = down66[down66['sigma-�00'].between(23.6,25.3)]
p1_66_pcm.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/RR1804/P1_2018/primary_chl/Station33_cast66_PCM.csv")

# get the sum of primary chl slice oxygen and beam transmission
cf_sum_p1_66_pcm = p1_66_pcm['flSP'].sum()
print('PCM Chl fluor sum:', cf_sum_p1_66_pcm)

bt_sum_p1_66_pcm = p1_66_pcm['CStarTr0'].sum()
print('PCM Beam trans sum:', bt_sum_p1_66_pcm)

o2_sum_p1_66_pcm = p1_66_pcm['sbox0Mm/Kg'].sum()
print('PCM O2 Mm/Kg sum:', o2_sum_p1_66_pcm)

# Make lists of the input for the PCM integrated dataframe
PCM_data_66 = {'Cast':[66], 'StartTime':st66, 'StartTime matlab datenum': datetime_to_datenum(ST_66), \
            'Int chl fluor': cf_sum_p1_66_pcm, \
           'Int beam trans': bt_sum_p1_66_pcm, \
            'Int O2 Mm/Kg': o2_sum_p1_66_pcm}

# slice the deep chlorophyll max out and save as csv
p1_66_dcm = down66[down66['sigma-�00'].between(25.3,26.4)]
p1_66_dcm.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/RR1804/P1_2018/deep_chl/Station33_cast66_DCM.csv")

# get the sum of deep chl slice oxygen and beam transmission
cf_sum_p1_66_dcm = p1_66_dcm['flSP'].sum()
print('DCM Chl fluor sum:', cf_sum_p1_66_dcm)

bt_sum_p1_66_dcm = p1_66_dcm['CStarTr0'].sum()
print('DCM Beam trans sum:', bt_sum_p1_66_dcm)

o2_sum_p1_66_dcm = p1_66_dcm['sbox0Mm/Kg'].sum()
print('DCM O2 Mm/Kg sum:', o2_sum_p1_66_dcm)

# Make lists of the input for the DCM integrated dataframe
DCM_data_66 = {'Cast':[66], 'StartTime':st66, 'StartTime matlab datenum': datetime_to_datenum(ST_66), \
            'Int chl fluor': cf_sum_p1_66_dcm, \
           'Int beam trans': bt_sum_p1_66_dcm, \
            'Int O2 Mm/Kg': o2_sum_p1_66_dcm}

ML Chl fluor sum: 12.39225
ML Beam trans sum: 101.81920000000001
ML O2 Mm/Kg sum: 4523.965
PCM Chl fluor sum: 15.218790000000004
PCM Beam trans sum: 66.1878
PCM O2 Mm/Kg sum: 1003.7769999999999
DCM Chl fluor sum: 47.18198799999999
DCM Beam trans sum: 680.5393
DCM O2 Mm/Kg sum: 239.47299999999996


In [20]:
# read from cnv with 'ignore' added to depSM
cast67 = ctd.from_cnv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/RR1804/RR1803-St33-P1/Station33_cast67real.cnv")

# set up datetime to datenum 
ST_67 = pd.to_datetime(st67)

# split up and down casts; we want the downcast for the profile
down67, up67 = cast67.split()

# add a column with the start date and time in UTC (this is from the cnv header or from ship's log)
down67.insert(0, 'StartTime', pd.to_datetime(st67).replace(microsecond=0))

# slice the mixed layer out and save as csv
p1_67_ml = down67[down67['sigma-�00'].between(22.0,23.6)]
p1_67_ml.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/RR1804/P1_2018/mixed_layer/Station33_cast67_ML.csv")

# get the sum of mixed layer slice oxygen, beam transmission, and chlorophyll fluor
cf_sum_p1_67_ml = p1_67_ml['flSP'].sum()
print('ML Chl fluor sum:', cf_sum_p1_67_ml)

bt_sum_p1_67_ml = p1_67_ml['CStarTr0'].sum()
print('ML Beam trans sum:', bt_sum_p1_67_ml)

o2_sum_p1_67_ml = p1_67_ml['sbox0Mm/Kg'].sum()
print('ML O2 Mm/Kg sum:', o2_sum_p1_67_ml)

# Make lists of the input for the ML integrated dataframe
ML_data_67 = {'Cast':[67], 'StartTime':st67, 'StartTime matlab datenum': datetime_to_datenum(ST_67), \
           'Int chl fluor': cf_sum_p1_67_ml, \
           'Int beam trans': bt_sum_p1_67_ml, \
            'Int O2 Mm/Kg': o2_sum_p1_67_ml}

# slice the primary chlorophyll max out and save as csv
p1_67_pcm = down67[down67['sigma-�00'].between(23.6,25.3)]
p1_67_pcm.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/RR1804/P1_2018/primary_chl/Station33_cast67_PCM.csv")

# get the sum of primary chl slice oxygen and beam transmission
cf_sum_p1_67_pcm = p1_67_pcm['flSP'].sum()
print('PCM Chl fluor sum:', cf_sum_p1_67_pcm)

bt_sum_p1_67_pcm = p1_67_pcm['CStarTr0'].sum()
print('PCM Beam trans sum:', bt_sum_p1_67_pcm)

o2_sum_p1_67_pcm = p1_67_pcm['sbox0Mm/Kg'].sum()
print('PCM O2 Mm/Kg sum:', o2_sum_p1_67_pcm)

# Make lists of the input for the PCM integrated dataframe
PCM_data_67 = {'Cast':[67], 'StartTime':st67, 'StartTime matlab datenum': datetime_to_datenum(ST_67), \
            'Int chl fluor': cf_sum_p1_67_pcm, \
           'Int beam trans': bt_sum_p1_67_pcm, \
            'Int O2 Mm/Kg': o2_sum_p1_67_pcm}

# slice the deep chlorophyll max out and save as csv
p1_67_dcm = down67[down67['sigma-�00'].between(25.3,26.4)]
p1_67_dcm.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/RR1804/P1_2018/deep_chl/Station33_cast67_DCM.csv")

# get the sum of deep chl slice oxygen and beam transmission
cf_sum_p1_67_dcm = p1_67_dcm['flSP'].sum()
print('DCM Chl fluor sum:', cf_sum_p1_67_dcm)

bt_sum_p1_67_dcm = p1_67_dcm['CStarTr0'].sum()
print('DCM Beam trans sum:', bt_sum_p1_67_dcm)

o2_sum_p1_67_dcm = p1_67_dcm['sbox0Mm/Kg'].sum()
print('DCM O2 Mm/Kg sum:', o2_sum_p1_67_dcm)

# Make lists of the input for the DCM integrated dataframe
DCM_data_67 = {'Cast':[67], 'StartTime':st67, 'StartTime matlab datenum': datetime_to_datenum(ST_67), \
            'Int chl fluor': cf_sum_p1_67_dcm, \
           'Int beam trans': bt_sum_p1_67_dcm, \
            'Int O2 Mm/Kg': o2_sum_p1_67_dcm}

ML Chl fluor sum: 13.219180000000001
ML Beam trans sum: 83.3297
ML O2 Mm/Kg sum: 3771.701
PCM Chl fluor sum: 24.27282
PCM Beam trans sum: 83.2942
PCM O2 Mm/Kg sum: 873.1250000000001
DCM Chl fluor sum: 50.791788
DCM Beam trans sum: 516.5119
DCM O2 Mm/Kg sum: 184.123


In [21]:
# read from cnv with 'ignore' added to depSM
cast68 = ctd.from_cnv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/RR1804/RR1803-St33-P1/Station33_cast68.cnv")

# set up datetime to datenum 
ST_68 = pd.to_datetime(st68)

# split up and down casts; we want the downcast for the profile
down68, up68 = cast68.split()

# add a column with the start date and time in UTC (this is from the cnv header or from ship's log)
down68.insert(0, 'StartTime', pd.to_datetime(st68).replace(microsecond=0))

# slice the mixed layer out and save as csv
p1_68_ml = down68[down68['sigma-�00'].between(22.0,23.6)]
p1_68_ml.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/RR1804/P1_2018/mixed_layer/Station33_cast68_ML.csv")

# get the sum of mixed layer slice oxygen, beam transmission, and chlorophyll fluor
cf_sum_p1_68_ml = p1_68_ml['flSP'].sum()
print('ML Chl fluor sum:', cf_sum_p1_68_ml)

bt_sum_p1_68_ml = p1_68_ml['CStarTr0'].sum()
print('ML Beam trans sum:', bt_sum_p1_68_ml)

o2_sum_p1_68_ml = p1_68_ml['sbox0Mm/Kg'].sum()
print('ML O2 Mm/Kg sum:', o2_sum_p1_68_ml)

# Make lists of the input for the ML integrated dataframe
ML_data_68 = {'Cast':[68], 'StartTime':st68, 'StartTime matlab datenum': datetime_to_datenum(ST_68), \
           'Int chl fluor': cf_sum_p1_68_ml, \
           'Int beam trans': bt_sum_p1_68_ml, \
            'Int O2 Mm/Kg': o2_sum_p1_68_ml}

# slice the primary chlorophyll max out and save as csv
p1_68_pcm = down68[down68['sigma-�00'].between(23.6,25.3)]
p1_68_pcm.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/RR1804/P1_2018/primary_chl/Station33_cast68_PCM.csv")

# get the sum of primary chl slice oxygen and beam transmission
cf_sum_p1_68_pcm = p1_68_pcm['flSP'].sum()
print('PCM Chl fluor sum:', cf_sum_p1_68_pcm)

bt_sum_p1_68_pcm = p1_68_pcm['CStarTr0'].sum()
print('PCM Beam trans sum:', bt_sum_p1_68_pcm)

o2_sum_p1_68_pcm = p1_68_pcm['sbox0Mm/Kg'].sum()
print('PCM O2 Mm/Kg sum:', o2_sum_p1_68_pcm)

# Make lists of the input for the PCM integrated dataframe
PCM_data_68 = {'Cast':[68], 'StartTime':st68, 'StartTime matlab datenum': datetime_to_datenum(ST_68), \
            'Int chl fluor': cf_sum_p1_68_pcm, \
           'Int beam trans': bt_sum_p1_68_pcm, \
            'Int O2 Mm/Kg': o2_sum_p1_68_pcm}

# slice the deep chlorophyll max out and save as csv
p1_68_dcm = down68[down68['sigma-�00'].between(25.3,26.4)]
p1_68_dcm.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/RR1804/P1_2018/deep_chl/Station33_cast68_DCM.csv")

# get the sum of deep chl slice oxygen and beam transmission
cf_sum_p1_68_dcm = p1_68_dcm['flSP'].sum()
print('DCM Chl fluor sum:', cf_sum_p1_68_dcm)

bt_sum_p1_68_dcm = p1_68_dcm['CStarTr0'].sum()
print('DCM Beam trans sum:', bt_sum_p1_68_dcm)

o2_sum_p1_68_dcm = p1_68_dcm['sbox0Mm/Kg'].sum()
print('DCM O2 Mm/Kg sum:', o2_sum_p1_68_dcm)

# Make lists of the input for the DCM integrated dataframe
DCM_data_68 = {'Cast':[68], 'StartTime':st68, 'StartTime matlab datenum': datetime_to_datenum(ST_68), \
            'Int chl fluor': cf_sum_p1_68_dcm, \
           'Int beam trans': bt_sum_p1_68_dcm, \
            'Int O2 Mm/Kg': o2_sum_p1_68_dcm}

ML Chl fluor sum: 19.402829999999998
ML Beam trans sum: 92.56700000000001
ML O2 Mm/Kg sum: 4126.806
PCM Chl fluor sum: 20.21198
PCM Beam trans sum: 75.2698
PCM O2 Mm/Kg sum: 1035.953
DCM Chl fluor sum: 41.690963
DCM Beam trans sum: 666.8783999999999
DCM O2 Mm/Kg sum: 232.44899999999998


In [22]:
# read from cnv with 'ignore' added to depSM
cast69 = ctd.from_cnv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/RR1804/RR1803-St33-P1/Station33_cast69.cnv")

# set up datetime to datenum 
ST_69 = pd.to_datetime(st69)

# split up and down casts; we want the downcast for the profile
down69, up69 = cast69.split()

# add a column with the start date and time in UTC (this is from the cnv header or from ship's log)
down69.insert(0, 'StartTime', pd.to_datetime(st69).replace(microsecond=0))

# slice the mixed layer out and save as csv
p1_69_ml = down69[down69['sigma-�00'].between(22.0,23.6)]
p1_69_ml.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/RR1804/P1_2018/mixed_layer/Station33_cast69_ML.csv")

# get the sum of mixed layer slice oxygen, beam transmission, and chlorophyll fluor
cf_sum_p1_69_ml = p1_69_ml['flSP'].sum()
print('ML Chl fluor sum:', cf_sum_p1_69_ml)

bt_sum_p1_69_ml = p1_69_ml['CStarTr0'].sum()
print('ML Beam trans sum:', bt_sum_p1_69_ml)

o2_sum_p1_69_ml = p1_69_ml['sbox0Mm/Kg'].sum()
print('ML O2 Mm/Kg sum:', o2_sum_p1_69_ml)

# Make lists of the input for the ML integrated dataframe
ML_data_69 = {'Cast':[69], 'StartTime':st69, 'StartTime matlab datenum': datetime_to_datenum(ST_69), \
           'Int chl fluor': cf_sum_p1_69_ml, \
           'Int beam trans': bt_sum_p1_69_ml, \
            'Int O2 Mm/Kg': o2_sum_p1_69_ml}

# slice the primary chlorophyll max out and save as csv
p1_69_pcm = down69[down69['sigma-�00'].between(23.6,25.3)]
p1_69_pcm.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/RR1804/P1_2018/primary_chl/Station33_cast69_PCM.csv")

# get the sum of primary chl slice oxygen and beam transmission
cf_sum_p1_69_pcm = p1_69_pcm['flSP'].sum()
print('PCM Chl fluor sum:', cf_sum_p1_69_pcm)

bt_sum_p1_69_pcm = p1_69_pcm['CStarTr0'].sum()
print('PCM Beam trans sum:', bt_sum_p1_69_pcm)

o2_sum_p1_69_pcm = p1_69_pcm['sbox0Mm/Kg'].sum()
print('PCM O2 Mm/Kg sum:', o2_sum_p1_69_pcm)

# Make lists of the input for the PCM integrated dataframe
PCM_data_69 = {'Cast':[69], 'StartTime':st69, 'StartTime matlab datenum': datetime_to_datenum(ST_69), \
            'Int chl fluor': cf_sum_p1_69_pcm, \
           'Int beam trans': bt_sum_p1_69_pcm, \
            'Int O2 Mm/Kg': o2_sum_p1_69_pcm}

# slice the deep chlorophyll max out and save as csv
p1_69_dcm = down69[down69['sigma-�00'].between(25.3,26.4)]
p1_69_dcm.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/RR1804/P1_2018/deep_chl/Station33_cast69_DCM.csv")

# get the sum of deep chl slice oxygen and beam transmission
cf_sum_p1_69_dcm = p1_69_dcm['flSP'].sum()
print('DCM Chl fluor sum:', cf_sum_p1_69_dcm)

bt_sum_p1_69_dcm = p1_69_dcm['CStarTr0'].sum()
print('DCM Beam trans sum:', bt_sum_p1_69_dcm)

o2_sum_p1_69_dcm = p1_69_dcm['sbox0Mm/Kg'].sum()
print('DCM O2 Mm/Kg sum:', o2_sum_p1_69_dcm)

# Make lists of the input for the DCM integrated dataframe
DCM_data_69 = {'Cast':[69], 'StartTime':st69, 'StartTime matlab datenum': datetime_to_datenum(ST_69), \
            'Int chl fluor': cf_sum_p1_69_dcm, \
           'Int beam trans': bt_sum_p1_69_dcm, \
            'Int O2 Mm/Kg': o2_sum_p1_69_dcm}

ML Chl fluor sum: 11.817599999999999
ML Beam trans sum: 79.5293
ML O2 Mm/Kg sum: 3513.0489999999995
PCM Chl fluor sum: 19.02719
PCM Beam trans sum: 93.48889999999999
PCM O2 Mm/Kg sum: 1452.255
DCM Chl fluor sum: 44.024456
DCM Beam trans sum: 730.3894
DCM O2 Mm/Kg sum: 230.957


In [23]:
# read from cnv with 'ignore' added to depSM
cast70 = ctd.from_cnv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/RR1804/RR1803-St33-P1/Station33_cast70.cnv")

# set up datetime to datenum 
ST_70 = pd.to_datetime(st70)

# split up and down casts; we want the downcast for the profile
down70, up70 = cast70.split()

# add a column with the start date and time in UTC (this is from the cnv header or from ship's log)
down70.insert(0, 'StartTime', pd.to_datetime(st70).replace(microsecond=0))

# slice the mixed layer out and save as csv
p1_70_ml = down70[down70['sigma-�00'].between(22.0,23.6)]
p1_70_ml.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/RR1804/P1_2018/mixed_layer/Station33_cast70_ML.csv")

# get the sum of mixed layer slice oxygen, beam transmission, and chlorophyll fluor
cf_sum_p1_70_ml = p1_70_ml['flSP'].sum()
print('ML Chl fluor sum:', cf_sum_p1_70_ml)

bt_sum_p1_70_ml = p1_70_ml['CStarTr0'].sum()
print('ML Beam trans sum:', bt_sum_p1_70_ml)

o2_sum_p1_70_ml = p1_70_ml['sbox0Mm/Kg'].sum()
print('ML O2 Mm/Kg sum:', o2_sum_p1_70_ml)

# Make lists of the input for the ML integrated dataframe
ML_data_70 = {'Cast':[70], 'StartTime':st70, 'StartTime matlab datenum': datetime_to_datenum(ST_70), \
           'Int chl fluor': cf_sum_p1_70_ml, \
           'Int beam trans': bt_sum_p1_70_ml, \
            'Int O2 Mm/Kg': o2_sum_p1_70_ml}

# slice the primary chlorophyll max out and save as csv
p1_70_pcm = down70[down70['sigma-�00'].between(23.6,25.3)]
p1_70_pcm.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/RR1804/P1_2018/primary_chl/Station33_cast70_PCM.csv")

# get the sum of primary chl slice oxygen and beam transmission
cf_sum_p1_70_pcm = p1_70_pcm['flSP'].sum()
print('PCM Chl fluor sum:', cf_sum_p1_70_pcm)

bt_sum_p1_70_pcm = p1_70_pcm['CStarTr0'].sum()
print('PCM Beam trans sum:', bt_sum_p1_70_pcm)

o2_sum_p1_70_pcm = p1_70_pcm['sbox0Mm/Kg'].sum()
print('PCM O2 Mm/Kg sum:', o2_sum_p1_70_pcm)

# Make lists of the input for the PCM integrated dataframe
PCM_data_70 = {'Cast':[70], 'StartTime':st70, 'StartTime matlab datenum': datetime_to_datenum(ST_70), \
            'Int chl fluor': cf_sum_p1_70_pcm, \
           'Int beam trans': bt_sum_p1_70_pcm, \
            'Int O2 Mm/Kg': o2_sum_p1_70_pcm}

# slice the deep chlorophyll max out and save as csv
p1_70_dcm = down70[down70['sigma-�00'].between(25.3,26.4)]
p1_70_dcm.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/RR1804/P1_2018/deep_chl/Station33_cast70_DCM.csv")

# get the sum of deep chl slice oxygen and beam transmission
cf_sum_p1_70_dcm = p1_70_dcm['flSP'].sum()
print('DCM Chl fluor sum:', cf_sum_p1_70_dcm)

bt_sum_p1_70_dcm = p1_70_dcm['CStarTr0'].sum()
print('DCM Beam trans sum:', bt_sum_p1_70_dcm)

o2_sum_p1_70_dcm = p1_70_dcm['sbox0Mm/Kg'].sum()
print('DCM O2 Mm/Kg sum:', o2_sum_p1_70_dcm)

# Make lists of the input for the DCM integrated dataframe
DCM_data_70 = {'Cast':[70], 'StartTime':st70, 'StartTime matlab datenum': datetime_to_datenum(ST_70), \
            'Int chl fluor': cf_sum_p1_70_dcm, \
           'Int beam trans': bt_sum_p1_70_dcm, \
            'Int O2 Mm/Kg': o2_sum_p1_70_dcm}

ML Chl fluor sum: 15.198129999999999
ML Beam trans sum: 96.91890000000001
ML O2 Mm/Kg sum: 4351.2519999999995
PCM Chl fluor sum: 24.764110000000002
PCM Beam trans sum: 96.72840000000001
PCM O2 Mm/Kg sum: 1292.336
DCM Chl fluor sum: 46.447522
DCM Beam trans sum: 707.7860000000001
DCM O2 Mm/Kg sum: 199.52799999999996


In [24]:
# read from cnv with 'ignore' added to depSM
cast71 = ctd.from_cnv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/RR1804/RR1803-St33-P1/Station33_cast71.cnv")

# set up datetime to datenum 
ST_71 = pd.to_datetime(st71)

# split up and down casts; we want the downcast for the profile
down71, up71 = cast71.split()

# add a column with the start date and time in UTC (this is from the cnv header or from ship's log)
down71.insert(0, 'StartTime', pd.to_datetime(st71).replace(microsecond=0))

# slice the mixed layer out and save as csv
p1_71_ml = down71[down71['sigma-�00'].between(22.0,23.6)]
p1_71_ml.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/RR1804/P1_2018/mixed_layer/Station33_cast71_ML.csv")

# get the sum of mixed layer slice oxygen, beam transmission, and chlorophyll fluor
cf_sum_p1_71_ml = p1_71_ml['flSP'].sum()
print('ML Chl fluor sum:', cf_sum_p1_71_ml)

bt_sum_p1_71_ml = p1_71_ml['CStarTr0'].sum()
print('ML Beam trans sum:', bt_sum_p1_71_ml)

o2_sum_p1_71_ml = p1_71_ml['sbox0Mm/Kg'].sum()
print('ML O2 Mm/Kg sum:', o2_sum_p1_71_ml)

# Make lists of the input for the ML integrated dataframe
ML_data_71 = {'Cast':[71], 'StartTime':st71, 'StartTime matlab datenum': datetime_to_datenum(ST_71), \
           'Int chl fluor': cf_sum_p1_71_ml, \
           'Int beam trans': bt_sum_p1_71_ml, \
            'Int O2 Mm/Kg': o2_sum_p1_71_ml}

# slice the primary chlorophyll max out and save as csv
p1_71_pcm = down71[down71['sigma-�00'].between(23.6,25.3)]
p1_71_pcm.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/RR1804/P1_2018/primary_chl/Station33_cast71_PCM.csv")

# get the sum of primary chl slice oxygen and beam transmission
cf_sum_p1_71_pcm = p1_71_pcm['flSP'].sum()
print('PCM Chl fluor sum:', cf_sum_p1_71_pcm)

bt_sum_p1_71_pcm = p1_71_pcm['CStarTr0'].sum()
print('PCM Beam trans sum:', bt_sum_p1_71_pcm)

o2_sum_p1_71_pcm = p1_71_pcm['sbox0Mm/Kg'].sum()
print('PCM O2 Mm/Kg sum:', o2_sum_p1_71_pcm)

# Make lists of the input for the PCM integrated dataframe
PCM_data_71 = {'Cast':[71], 'StartTime':st71, 'StartTime matlab datenum': datetime_to_datenum(ST_71), \
            'Int chl fluor': cf_sum_p1_71_pcm, \
           'Int beam trans': bt_sum_p1_71_pcm, \
            'Int O2 Mm/Kg': o2_sum_p1_71_pcm}

# slice the deep chlorophyll max out and save as csv
p1_71_dcm = down71[down71['sigma-�00'].between(25.3,26.4)]
p1_71_dcm.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/RR1804/P1_2018/deep_chl/Station33_cast71_DCM.csv")

# get the sum of deep chl slice oxygen and beam transmission
cf_sum_p1_71_dcm = p1_71_dcm['flSP'].sum()
print('DCM Chl fluor sum:', cf_sum_p1_71_dcm)

bt_sum_p1_71_dcm = p1_71_dcm['CStarTr0'].sum()
print('DCM Beam trans sum:', bt_sum_p1_71_dcm)

o2_sum_p1_71_dcm = p1_71_dcm['sbox0Mm/Kg'].sum()
print('DCM O2 Mm/Kg sum:', o2_sum_p1_71_dcm)

# Make lists of the input for the DCM integrated dataframe
DCM_data_71 = {'Cast':[71], 'StartTime':st71, 'StartTime matlab datenum': datetime_to_datenum(ST_71), \
            'Int chl fluor': cf_sum_p1_71_dcm, \
           'Int beam trans': bt_sum_p1_71_dcm, \
            'Int O2 Mm/Kg': o2_sum_p1_71_dcm}

ML Chl fluor sum: 12.89634
ML Beam trans sum: 74.30730000000001
ML O2 Mm/Kg sum: 3370.414
PCM Chl fluor sum: 31.35596
PCM Beam trans sum: 104.9205
PCM O2 Mm/Kg sum: 1746.7540000000001
DCM Chl fluor sum: 57.458093
DCM Beam trans sum: 706.8835999999999
DCM O2 Mm/Kg sum: 207.39399999999998


In [25]:
# read from cnv with 'ignore' added to depSM
cast72 = ctd.from_cnv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/RR1804/RR1803-St33-P1/Station33_cast72.cnv")

# set up datetime to datenum 
ST_72 = pd.to_datetime(st72)

# split up and down casts; we want the downcast for the profile
down72, up72 = cast72.split()

# add a column with the start date and time in UTC (this is from the cnv header or from ship's log)
down72.insert(0, 'StartTime', pd.to_datetime(st72).replace(microsecond=0))

# slice the mixed layer out and save as csv
p1_72_ml = down72[down72['sigma-�00'].between(22.0,23.6)]
p1_72_ml.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/RR1804/P1_2018/mixed_layer/Station33_cast72_ML.csv")

# get the sum of mixed layer slice oxygen, beam transmission, and chlorophyll fluor
cf_sum_p1_72_ml = p1_72_ml['flSP'].sum()
print('ML Chl fluor sum:', cf_sum_p1_72_ml)

bt_sum_p1_72_ml = p1_72_ml['CStarTr0'].sum()
print('ML Beam trans sum:', bt_sum_p1_72_ml)

o2_sum_p1_72_ml = p1_72_ml['sbox0Mm/Kg'].sum()
print('ML O2 Mm/Kg sum:', o2_sum_p1_72_ml)

# Make lists of the input for the ML integrated dataframe
ML_data_72 = {'Cast':[72], 'StartTime':st72, 'StartTime matlab datenum': datetime_to_datenum(ST_72), \
           'Int chl fluor': cf_sum_p1_72_ml, \
           'Int beam trans': bt_sum_p1_72_ml, \
            'Int O2 Mm/Kg': o2_sum_p1_72_ml}

# slice the primary chlorophyll max out and save as csv
p1_72_pcm = down72[down72['sigma-�00'].between(23.6,25.3)]
p1_72_pcm.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/RR1804/P1_2018/primary_chl/Station33_cast72_PCM.csv")

# get the sum of primary chl slice oxygen and beam transmission
cf_sum_p1_72_pcm = p1_72_pcm['flSP'].sum()
print('PCM Chl fluor sum:', cf_sum_p1_72_pcm)

bt_sum_p1_72_pcm = p1_72_pcm['CStarTr0'].sum()
print('PCM Beam trans sum:', bt_sum_p1_72_pcm)

o2_sum_p1_72_pcm = p1_72_pcm['sbox0Mm/Kg'].sum()
print('PCM O2 Mm/Kg sum:', o2_sum_p1_72_pcm)

# Make lists of the input for the PCM integrated dataframe
PCM_data_72 = {'Cast':[72], 'StartTime':st72, 'StartTime matlab datenum': datetime_to_datenum(ST_72), \
            'Int chl fluor': cf_sum_p1_72_pcm, \
           'Int beam trans': bt_sum_p1_72_pcm, \
            'Int O2 Mm/Kg': o2_sum_p1_72_pcm}

# slice the deep chlorophyll max out and save as csv
p1_72_dcm = down72[down72['sigma-�00'].between(25.3,26.4)]
p1_72_dcm.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/RR1804/P1_2018/deep_chl/Station33_cast72_DCM.csv")

# get the sum of deep chl slice oxygen and beam transmission
cf_sum_p1_72_dcm = p1_72_dcm['flSP'].sum()
print('DCM Chl fluor sum:', cf_sum_p1_72_dcm)

bt_sum_p1_72_dcm = p1_72_dcm['CStarTr0'].sum()
print('DCM Beam trans sum:', bt_sum_p1_72_dcm)

o2_sum_p1_72_dcm = p1_72_dcm['sbox0Mm/Kg'].sum()
print('DCM O2 Mm/Kg sum:', o2_sum_p1_72_dcm)

# Make lists of the input for the DCM integrated dataframe
DCM_data_72 = {'Cast':[72], 'StartTime':st72, 'StartTime matlab datenum': datetime_to_datenum(ST_72), \
            'Int chl fluor': cf_sum_p1_72_dcm, \
           'Int beam trans': bt_sum_p1_72_dcm, \
            'Int O2 Mm/Kg': o2_sum_p1_72_dcm}

ML Chl fluor sum: 12.06379
ML Beam trans sum: 93.6515
ML O2 Mm/Kg sum: 4055.4559999999997
PCM Chl fluor sum: 22.4816
PCM Beam trans sum: 74.9124
PCM O2 Mm/Kg sum: 773.291
DCM Chl fluor sum: 55.444939000000005
DCM Beam trans sum: 716.5942000000001
DCM O2 Mm/Kg sum: 239.933


In [26]:
#ML_2018_p1.loc[len(ML_2018_p1)] = ML_data_43
ML_2018_p1 = ML_2018_p1.append(ML_data_59, ignore_index=True)
ML_2018_p1 = ML_2018_p1.append(ML_data_60, ignore_index=True)
ML_2018_p1 = ML_2018_p1.append(ML_data_61, ignore_index=True)
ML_2018_p1 = ML_2018_p1.append(ML_data_62, ignore_index=True)
ML_2018_p1 = ML_2018_p1.append(ML_data_63, ignore_index=True)
ML_2018_p1 = ML_2018_p1.append(ML_data_64, ignore_index=True)
ML_2018_p1 = ML_2018_p1.append(ML_data_65, ignore_index=True)
ML_2018_p1 = ML_2018_p1.append(ML_data_66, ignore_index=True)
ML_2018_p1 = ML_2018_p1.append(ML_data_67, ignore_index=True)
ML_2018_p1 = ML_2018_p1.append(ML_data_68, ignore_index=True)
ML_2018_p1 = ML_2018_p1.append(ML_data_69, ignore_index=True)
ML_2018_p1 = ML_2018_p1.append(ML_data_70, ignore_index=True)
ML_2018_p1 = ML_2018_p1.append(ML_data_71, ignore_index=True)
ML_2018_p1 = ML_2018_p1.append(ML_data_72, ignore_index=True)

ML_2018_p1.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/RR1804/P1_2018/mixed_layer/Station33_integrated_ML.csv")

ML_2018_p1.head(16)

,Cast,StartTime,StartTime matlab datenum,Int chl fluor,Int beam trans,Int O2 Mm/Kg
0,59,2018-04-21 15:39:15,743856.652257,0.00000,0.0000,0.000
1,[59],2018-04-21 15:39:15,743856.652257,0.00000,0.0000,0.000
2,[60],2018-04-22 10:12:45,743858.425521,13.59100,43.1305,1916.164
3,[61],2018-04-22 14:04:25,743858.586400,34.94350,76.2150,3480.805
4,[62],2018-04-22 22:00:31,743858.917025,20.38190,41.7564,1928.755
5,[63],2018-04-23 14:00:59,743860.584016,15.83736,47.4445,2159.679
6,[64],2018-04-24 14:03:01,743862.585428,18.95489,100.6623,4508.090
7,[65],2018-04-24 21:04:47,743862.878322,17.39061,83.0200,3700.612
8,[66],2018-04-25 15:02:53,743864.627002,12.39225,101.8192,4523.965
9,[67],2018-04-26 00:07:04,743866.004907,13.21918,83.3297,3771.701


In [27]:
#PCM_2018_p1.loc[len(PCM_2018_p1)] = PCM_data_43
PCM_2018_p1 = PCM_2018_p1.append(PCM_data_59, ignore_index=True)
PCM_2018_p1 = PCM_2018_p1.append(PCM_data_60, ignore_index=True)
PCM_2018_p1 = PCM_2018_p1.append(PCM_data_61, ignore_index=True)
PCM_2018_p1 = PCM_2018_p1.append(PCM_data_62, ignore_index=True)
PCM_2018_p1 = PCM_2018_p1.append(PCM_data_63, ignore_index=True)
PCM_2018_p1 = PCM_2018_p1.append(PCM_data_64, ignore_index=True)
PCM_2018_p1 = PCM_2018_p1.append(PCM_data_65, ignore_index=True)
PCM_2018_p1 = PCM_2018_p1.append(PCM_data_66, ignore_index=True)
PCM_2018_p1 = PCM_2018_p1.append(PCM_data_67, ignore_index=True)
PCM_2018_p1 = PCM_2018_p1.append(PCM_data_68, ignore_index=True)
PCM_2018_p1 = PCM_2018_p1.append(PCM_data_69, ignore_index=True)
PCM_2018_p1 = PCM_2018_p1.append(PCM_data_70, ignore_index=True)
PCM_2018_p1 = PCM_2018_p1.append(PCM_data_71, ignore_index=True)
PCM_2018_p1 = PCM_2018_p1.append(PCM_data_72, ignore_index=True)

PCM_2018_p1.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/RR1804/P1_2018/primary_chl/Station33_integrated_PCM.csv")

PCM_2018_p1.head(17)

,Cast,StartTime,StartTime matlab datenum,Int chl fluor,Int beam trans,Int O2 Mm/Kg
0,59,2018-04-21 15:39:15,743856.652257,61.12623,112.6947,2577.237
1,[59],2018-04-21 15:39:15,743856.652257,61.12623,112.6947,2577.237
2,[60],2018-04-22 10:12:45,743858.425521,29.76870,141.6178,2662.933
3,[61],2018-04-22 14:04:25,743858.586400,15.67199,75.1618,1328.632
4,[62],2018-04-22 22:00:31,743858.917025,29.96295,109.6267,2251.013
5,[63],2018-04-23 14:00:59,743860.584016,17.77299,93.3572,1393.820
6,[64],2018-04-24 14:03:01,743862.585428,15.61641,88.9701,1464.192
7,[65],2018-04-24 21:04:47,743862.878322,24.46099,93.0222,1453.772
8,[66],2018-04-25 15:02:53,743864.627002,15.21879,66.1878,1003.777
9,[67],2018-04-26 00:07:04,743866.004907,24.27282,83.2942,873.125


In [32]:
#DCM_2018_p1.loc[len(DCM_2018_p1)] = DCM_data_43
DCM_2018_p1 = DCM_2018_p1.append(DCM_data_59, ignore_index=True)
DCM_2018_p1 = DCM_2018_p1.append(DCM_data_60, ignore_index=True)
DCM_2018_p1 = DCM_2018_p1.append(DCM_data_61, ignore_index=True)
DCM_2018_p1 = DCM_2018_p1.append(DCM_data_62, ignore_index=True)
DCM_2018_p1 = DCM_2018_p1.append(DCM_data_63, ignore_index=True)
DCM_2018_p1 = DCM_2018_p1.append(DCM_data_64, ignore_index=True)
DCM_2018_p1 = DCM_2018_p1.append(DCM_data_, ignore_index=True)
DCM_2018_p1 = DCM_2018_p1.append(DCM_data_50, ignore_index=True)
DCM_2018_p1 = DCM_2018_p1.append(DCM_data_51, ignore_index=True)
DCM_2018_p1 = DCM_2018_p1.append(DCM_data_52, ignore_index=True)
DCM_2018_p1 = DCM_2018_p1.append(DCM_data_53, ignore_index=True)
DCM_2018_p1 = DCM_2018_p1.append(DCM_data_54, ignore_index=True)
DCM_2018_p1 = DCM_2018_p1.append(DCM_data_55, ignore_index=True)
DCM_2018_p1 = DCM_2018_p1.append(DCM_data_56, ignore_index=True)
DCM_2018_p1 = DCM_2018_p1.append(DCM_data_57, ignore_index=True)
DCM_2018_p1 = DCM_2018_p1.append(DCM_data_58, ignore_index=True)

DCM_2018_p1.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/RR1804/P1_2018/deep_chl/Station33_integrated_DCM.csv")

DCM_2018_p1.head(16)

,Cast,StartTime,StartTime matlab datenum,Int chl fluor,Int beam trans,Int O2 Mm/Kg
0,42,2018-04-15 05:12:33,743844.217049,15.92853,177.7587,45.691
1,[43],2018-04-15 15:06:02,743844.629190,15.70438,173.0129,47.779
2,[44],2018-04-15 22:01:43,743844.917859,14.91336,172.9927,51.398
3,[45],2018-04-16 15:03:02,743846.627106,14.53045,177.6250,46.232
4,[46],2018-04-17 15:03:16,743848.627269,11.04443,150.4668,43.161
5,[47],2018-04-17 21:04:08,743848.877870,11.92190,155.0208,49.977
6,[48],2018-04-18 15:03:03,743850.627118,12.29344,187.0308,65.163
7,[49],2018-04-18 22:05:25,743850.920428,10.97383,136.6731,47.955
8,[50],2018-04-19 02:15:57,743852.094410,10.39415,136.7008,46.091
9,[51],2018-04-19 06:07:15,743852.255035,17.16247,177.6435,53.381
